# The Sentinel-1 class
*Author: Gregor Perich*

In this notebook, we'll take a closer look at the `Sentinel1` [class](https://github.com/EOA-team/eodal/blob/master/eodal/core/sensors/sentinel1.py) of `eodal`. The `Sentinel1` class has many methods and attributes specifically tailored to the data structure of the [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1) satellites of the European Space Agency (ESA). Therefore, we begin with a quick introduction of the Sentinel-1 (S1) satellites. 

## The Sentinel-1 satellites
S1 is a pair or satellites (Sentinel-1A and 1B) at an orbit of ca. 700 km. The satellites carry a [C-band SAR instrument](https://sentinel.esa.int/web/sentinel/missions/sentinel-1/instrument-payload), which features a Synthetic Aperture Radar (SAR). This active instrument allows them to penetrate cloud cover and deliver an "image" at any weather condition - also at night. The C-SAR has a [centre frequency of 5.4 Ghz, resulting in a wavelength of ca. 5.5 cm](https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-1-sar/sar-instrument). The C-SAR instrument has four different data acquisition modes as illustrated in the figure below. The primary data colleciton modes are IW over land and WV over ocean.

<img src="../images_for_text/S1_acquisition_%20modes.png" width="500" />

*Sentinel-1 C-SAR data acquisition modes. Figure from ESA.*

The C-SAR allows dual polarisation (HH+HV, VV+VH) as well as single polarisation VV and HH. The different data acquisition modes result in different swath widths and resolutions: 

<img src="../images_for_text/S1_resolution_swath.png" width="500" />

*The spatial resolution of the Sentinel-1 data products depends on the acquisition mode and the processing level. Figure from ESA.*

## Revisit frequency
The S1 satellites have an individual revisit frequency of ca. 12 days, combined they achieve ca. 6 days. Sadly, the S1B satellite [experienced a malfunction](https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-1/Mission_ends_for_Copernicus_Sentinel-1B_satellite) on the 23.12.2021, which put it out of action. Therefore, since that date, only data from the S1A satellite is available at the reduced temporal frequency of ca. 12 days. 



## Data products
ESA offers the following [data products](https://sentinel.esa.int/web/sentinel/missions/sentinel-1/data-products) for Sentinel-1: 
- **Level-0 Raw:** Raw data from the satellites that need to be processed before usage
- **Level-1 Single Look Complex (SLC):** georeferenced, focused SAR data preserving the [phase information](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/product-overview/interferometry) of the data
- **Level-1 Ground Range Detected (GRD):** focused SAR data projected to an Earth ellipsoid model. Phase information of the data is lost. 
- L**evel-2 Ocean (OCN):** Specific data format containing estimations of wind speed and direction over the oceans. 


## Get S1 data
Here, we'll walk you through the process of querying, downloading and briefly analysing S1 data using the `Sentinel1` class featured in `eodal`. This should give you an example of the `Sentinel1` class methods and attributes of `eodal`.

In [1]:
# import libraries and set paths

import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
from eodal.config import get_settings
from eodal.core.sensors.sentinel1 import Sentinel1  # native support for Sentinel1
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs
from typing import List
from pathlib import Path

base_dir: Path = Path('../')
data_dir: Path = Path(base_dir.joinpath("data"))
temp_dir: Path = Path(base_dir.joinpath("temp_files"))

# Define settings, s.t. we can access the data from MS planetary computer
Settings = get_settings()
Settings.USE_STAC = True

Currently, `eodal` supports the Ground Range Detected (GRD) and the [Radiometrically Terrain Corrected](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc) (RTC) products. The RTC is a further processed verison of the GRD product and is - as the name suggests - terrain corrected. This step is usually very knowledge and computationally intensive! Luckily, the data set is available on the MS planetary computer. However, it requires an API token. Alternatively, you can run this script directly on the MS planetary computer.  

Here, we load in an AOI from the beautiful city of Zürich. Zürich lies at lake Zürich. As water bodies are really easy to detect in SAR imagery, we take an AOI with a large lake. 

In [8]:
# -------------------------- Collection -------------------------------
# select Sentinel1 collection - GRD product
collection: str = "sentinel1-grd"

# ------------------------- Time Range ---------------------------------
time_start: datetime = datetime(2023, 5, 12)  # year, month, day (incl.)
time_end: datetime = datetime(2023, 6, 12)  # year, month, day (incl.)

# ---------------------- Spatial Feature  ------------------------------
geom: Path = Path(data_dir.joinpath("zurich_bbox.gpkg"))

# ------------------------- Metadata Filters ---------------------------
# select the RTC product
metadata_filters: List[Filter] = [
    Filter('product_type', '==', 'RTC')
]

# -------- Define kwargs for the scene constructor ---------------------
scene_kwargs = {
    "scene_constructor": Sentinel1.from_safe,
    "scene_modifier_kwargs": {}
}

# query the scenes available (no I/O of scenes, this only fetches metadata)
feature = Feature.from_geoseries(gpd.read_file(geom).geometry)
mapper_configs = MapperConfigs(
    collection=collection,
    time_start=time_start,
    time_end=time_end,
    feature=feature,
    metadata_filters=metadata_filters,
)

# create a Mapper instance
mapper = Mapper(mapper_configs)

# fetch metadata
mapper.query_scenes()

Now, let's look at the available scenes of our query by looking at the metadata. We immediately notice in the `platform` column, that we only have data from the S1A satellite, due to S1B being out of operations. We also notice that we have higher revisit frequencies due to the fact that Zürich is at a medium latitiude (~47°N).

In [9]:
mapper.metadata

,platform,s1:shape,proj:bbox,proj:epsg,proj:shape,end_datetime,constellation,s1:resolution,proj:transform,s1:datatake_id,...,sar:observation_direction,sar:pixel_spacing_azimuth,sar:looks_equivalent_number,s1:instrument_configuration_ID,sat:platform_international_designator,assets,sensing_time,sensing_date,epsg,geom
8,SENTINEL-1A,"[29028, 21185]","[371070.0, 5079860.0, 661350.0, 5291710.0]",32632,"[21185, 29028]",2023-05-16 17:16:05.589966+00:00,Sentinel-1,high,"[10.0, 0.0, 371070.0, 0.0, -10.0, 5291710.0, 0...",382802,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-05-16T17:15:53.090599Z,2023-05-16,32632,"POLYGON ((10.98146 45.87128, 10.98146 47.75328..."
7,SENTINEL-1A,"[29179, 21199]","[335300.0, 5154970.0, 627090.0, 5366960.0]",32632,"[21199, 29179]",2023-05-20 05:35:42.756575+00:00,Sentinel-1,high,"[10.0, 0.0, 335300.0, 0.0, -10.0, 5366960.0, 0...",383177,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-05-20T05:35:30.257402Z,2023-05-20,32632,"POLYGON ((10.64533 46.54730, 10.64533 48.42967..."
6,SENTINEL-1A,"[28692, 20628]","[222390.0, 5034600.0, 509310.0, 5240880.0]",32632,"[20628, 28692]",2023-05-21 17:24:12.163649+00:00,Sentinel-1,high,"[10.0, 0.0, 222390.0, 0.0, -10.0, 5240880.0, 0...",383353,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-05-21T17:23:59.664278Z,2023-05-21,32632,"POLYGON ((9.00965 45.42679, 9.00965 47.31140, ..."
5,SENTINEL-1A,"[28551, 20519]","[197310.0, 5202210.0, 482820.0, 5407400.0]",32632,"[20519, 28551]",2023-05-21 17:24:37.163892+00:00,Sentinel-1,high,"[10.0, 0.0, 197310.0, 0.0, -10.0, 5407400.0, 0...",383353,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-05-21T17:24:24.664520Z,2023-05-21,32632,"POLYGON ((8.67718 46.92314, 8.67718 48.80873, ..."
4,SENTINEL-1A,"[28879, 21160]","[371060.0, 5079840.0, 659850.0, 5291440.0]",32632,"[21160, 28879]",2023-05-28 17:16:06.128874+00:00,Sentinel-1,high,"[10.0, 0.0, 371060.0, 0.0, -10.0, 5291440.0, 0...",384131,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-05-28T17:15:53.629539Z,2023-05-28,32632,"POLYGON ((10.98146 45.87128, 10.98146 47.75319..."
3,SENTINEL-1A,"[29180, 21199]","[335380.0, 5154970.0, 627180.0, 5366960.0]",32632,"[21199, 29180]",2023-06-01 05:35:43.163072+00:00,Sentinel-1,high,"[10.0, 0.0, 335380.0, 0.0, -10.0, 5366960.0, 0...",384513,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-06-01T05:35:30.663897Z,2023-06-01,32632,"POLYGON ((10.64627 46.54739, 10.64627 48.42968..."
2,SENTINEL-1A,"[28691, 20628]","[222390.0, 5034610.0, 509300.0, 5240890.0]",32632,"[20628, 28691]",2023-06-02 17:24:12.635649+00:00,Sentinel-1,high,"[10.0, 0.0, 222390.0, 0.0, -10.0, 5240890.0, 0...",384682,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-06-02T17:24:00.136315Z,2023-06-02,32632,"POLYGON ((9.00965 45.42688, 9.00965 47.31149, ..."
1,SENTINEL-1A,"[28551, 20519]","[197310.0, 5202210.0, 482820.0, 5407400.0]",32632,"[20519, 28551]",2023-06-02 17:24:37.635816+00:00,Sentinel-1,high,"[10.0, 0.0, 197310.0, 0.0, -10.0, 5407400.0, 0...",384682,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-06-02T17:24:25.136482Z,2023-06-02,32632,"POLYGON ((8.67717 46.92323, 8.67717 48.80882, ..."
0,SENTINEL-1A,"[28882, 21159]","[371140.0, 5079850.0, 659960.0, 5291440.0]",32632,"[21159, 28882]",2023-06-09 17:16:06.413876+00:00,Sentinel-1,high,"[10.0, 0.0, 371140.0, 0.0, -10.0, 5291440.0, 0...",385478,...,right,10,4.4,7,2014-016A,{'vh': {'href': 'https://sentinel1euwestrtc.bl...,2023-06-09T17:15:53.914521Z,2023-06-09,32632,"POLYGON ((10.98249 45.87111, 10.98249 47.75317..."


In [10]:
# get actual data, this is the I/O step
mapper.load_scenes(scene_kwargs=scene_kwargs)

# store the data (type = SceneCollection) separately
scene_coll = mapper.data

2023-06-14 13:28:28,114 eodal        INFO     Starting extraction of sentinel1 scenes


ValueError: Could not load scene:  403 Client Error: Forbidden for url: https://planetarycomputer.microsoft.com/api/sas/v1/token/sentinel1euwestrtc/sentinel1-grd-rtc